In [1]:
import wave
import time
import sys
import os
import zmq
import serial
import logging
import struct
import RPi.GPIO as GPIO

import pyaudio

In [11]:
GPIO.setmode(GPIO.BOARD)
class WavPlayer():
    def __init__(self, pin=5):
        self.pin = pin
        self.pa = pyaudio.PyAudio()
        self.wf = None
        self.played = False

        # init the pins
        GPIO.setup(self.pin, GPIO.OUT)
        GPIO.output(self.pin, GPIO.LOW)

    def play_callback(self, in_data, frame_count, time_info, status):
        data = self.wf.readframes(frame_count)
        return (data, pyaudio.paContinue)

    def play_file(self, wave_file_path):
        self.wf = wave.open(wave_file_path, 'rb')
        stream = self.pa.open(format=self.pa.get_format_from_width(self.wf.getsampwidth()),
                              channels=self.wf.getnchannels(),
                              rate=self.wf.getframerate(),
                              output=True,
                              stream_callback=self.play_callback)

        GPIO.output(self.pin, GPIO.HIGH)
        stream.start_stream()

        while stream.is_active():
            # time.sleep(0.1)
            pass
        GPIO.output(self.pin, GPIO.LOW)
        time.sleep(0.1)
        stream.stop_stream()
        stream.close()
        self.flush_file()

    def flush_file(self):
        self.wf = None
        self.played = False

In [12]:
pyaudio

<module 'pyaudio' from '/home/pi/.local/lib/python3.5/site-packages/pyaudio.py'>

In [13]:
import pyaudio
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  0  -  Steinberg UR22: USB Audio (hw:0,0)
Input Device id  1  -  sysdefault
Input Device id  5  -  spdif


In [20]:
wavefile_path = '/home/pi/repos/tone_test.wav'
wp = WavPlayer()
wp.play_file(wavefile_path)

In [ ]:
def run_trial(trial_pars: dict):
    # here is where the trial is defined
    # for now the trial is just playing a sound file
    # read the parameters
    # for the female presetation, you only want to do smartglass
    wavefile_path = trial_pars['stim_file']
    trial_number = int(float(trial_pars['number']))

    # do the deed
    so.write_number(trial_number)
    time.sleep(0.5)
    wp.play_file(wavefile_path)
    return 0